In [1]:
from dotenv import load_dotenv
from langchain_anthropic import ChatAnthropic
from sentence_transformers import SentenceTransformer
from utils.matching_functions import find_closest_documents

load_dotenv()
model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)
chat = ChatAnthropic(temperature=0, model_name="claude-3-haiku-20240307")

<All keys matched successfully>


In [2]:
from utils.wikipedia_functions import get_random_page, get_page_links, get_page_content, check_wikipedia_pages_existence

start_page, end_page = get_random_page(2)
end_page_full_content = get_page_content(end_page)
print(start_page)
print(end_page)

Water Lily Nebula
Portuguese Football Federation


In [3]:
start_links = get_page_links(start_page)
checked_links = check_wikipedia_pages_existence(start_links)
valid_links = [link for link in start_links if checked_links[link]]

In [4]:
from agents.chain_functions import get_crawler_chain, get_summarize_chain, get_explain_links_chain, get_broad_links_chain

crawler_chain = get_crawler_chain(chat)
summarize_chain = get_summarize_chain(chat)
explain_links_chain = get_explain_links_chain(chat)
broad_links_chain = get_broad_links_chain(chat)

In [5]:
from agents.answer_functions import answer_broad_links, answer_crawler, answer_explain_links, answer_summarize

end_page_content = answer_summarize(summarize_chain, end_page_full_content)
print(end_page_content)

The Portuguese Football Federation (FPF) is the governing body of football in Portugal, formed in 1914 as the Portuguese Football Union. It joined FIFA in 1923 and is a founding member of UEFA. The FPF oversees all aspects of football in Portugal, including professional, amateur, and national teams. It has gone through various leadership changes, including controversies like the "Saltillo Affair" in the 1986 World Cup. In recent years, the FPF has hosted major events like the 2019 UEFA Nations League Finals and is bidding to host the 2030 FIFA World Cup. The FPF is structured with various governing bodies, including the General Assembly, Board, Council of Justice, Disciplinary Board, Arbitration Council, and Fiscal Council.


In [7]:
current_page = start_page
current_links = valid_links
trajectory = []
for i in range(10):
    trajectory.append(current_page)
    if end_page in current_links:
        print(f"{end_page} in {current_page}'s links \nPage reached in {i+1} iterations!")
        break
    else:
        closest_links = find_closest_documents(model, current_links, end_page, end_page_content)
        broad_links = answer_broad_links(broad_links_chain, current_links, end_page, end_page_content)
        current_links = list(set([link for link in closest_links + broad_links if link in current_links]))
        reasonings = answer_explain_links(explain_links_chain, current_links, end_page, end_page_content)
        current_page, crawler_output = answer_crawler(crawler_chain, current_page, current_links, reasonings, end_page, end_page_content)
        print(f"Crawler Output: {crawler_output}")
        found_links = get_page_links(current_page)
        checked_links = check_wikipedia_pages_existence(found_links)
        current_links = [link for link in found_links if checked_links[link] and link not in trajectory]        

Crawler Output: <reasoning>
Based on the information provided, the link that seems most relevant to reaching the "Portuguese Football Federation" page is "Ara (constellation)". While this link may not have a direct connection to the target page, it could potentially provide some context about Portugal's geographical location and its relationship with other countries in the southern hemisphere. This information could be useful in understanding the broader context of the Portuguese Football Federation and its place within the global football landscape.
</reasoning>

<output>Ara (constellation)</output>
Crawler Output: <reasoning>
Based on the provided information, the most relevant link to click on to get to the goal page of the Portuguese Football Federation (FPF) appears to be the "International Astronomical Union (IAU)" link. While the connection may not be immediately obvious, the reasoning is that the IAU is an international governing body, similar to how the FPF is the governing bo